In [6]:
import base64
import os
import pickle
from io import BytesIO
from pathlib import Path

import dotenv
from azure.storage.blob import BlobServiceClient
import pandas as pd
from matplotlib.figure import Figure

In [7]:
def load_models():
    dotenv.load_dotenv()
    azure_storage_connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
    blob_service_client = BlobServiceClient.from_connection_string(
        azure_storage_connection_string
    )

    containers = blob_service_client.list_containers(include_metadata=True)
    suffix = 0
    for container in containers:
        existingContainerName = container["name"]
        if existingContainerName.startswith("energy-model"):
            parts = existingContainerName.split("-")
            newSuffix = int(parts[-1])
            if newSuffix > suffix:
                suffix = newSuffix

    container_name = f"energy-model-{suffix}"
    print(f"loading from container: {container_name}")

    container_client = blob_service_client.get_container_client(container_name)
    blob_list = container_client.list_blobs()

    models = dict()
    for blob in blob_list:
        print("\t" + blob.name)
        content = container_client.download_blob(blob.name).readall()
        model = pickle.loads(content)
        models[blob.name]=model
    return models

In [8]:
def plot_forecast(models, energy_types, forecast_horizon):
    """
    Compute the prediction for the given forecast_horizon
    Untransform the data, so that the original values are used
    Plot the untransformed data
    """
    plots = dict()
    forecast_dataset = dict()
    
    for name in energy_types:
        forecast_dataset[name] = models[f'{name.lower()}.pickle']
    
    for name, values in forecast_dataset.items():
        series = values.transformed_values   
        forecast = values.production_model.apply(series).get_forecast(steps=forecast_horizon)
        forecast_ci = forecast.conf_int()
    
    
        current = series.iloc[-1:]
        
        df = pd.DataFrame(dict(
            mean=pd.concat([current, forecast.predicted_mean]),
            low = pd.concat([current, forecast_ci.iloc[:,0]]),
            high = pd.concat([current, forecast_ci.iloc[:,1]]),
            )
        )
    
        untransform = lambda x: x if values.offset is None else 10**x-values.offset
    
        # Graph
        fig = Figure(figsize=(10, 6))
        ax = fig.subplots()
        ax.set_title(values.name)
            
        # Plot data points
        plot_start = series.index[-1] - pd.Timedelta(days=500)
        untransform(series.loc[plot_start:]).plot(ax=ax, label='Observed')  # Pandas plotting
    
        df_ut = untransform(df)
        
        # Plot predictions
        df_ut["mean"].plot(ax=ax, style='g', label=f'Forecast')
        ax.fill_between(df_ut.low.index, df_ut.low, df_ut.high, color='g', alpha=0.1)
        
        ax.legend(loc='lower right')
        ax.grid(True)
    
        buf = BytesIO()
        fig.savefig(buf, format="png", bbox_inches='tight')
        data = base64.b64encode(buf.getbuffer()).decode("ascii")
    
        plots[name] = f"data:image/png;base64,{data}"

    return plots

In [9]:
def run():
    models = load_models()
    print("models loaded from cloud")
    energy_types = ["wind", "Nuclear"]
    forecast_horizon = 30
    plot_forecast(models, energy_types, forecast_horizon)
    print("forecast plottet")